<a href="https://colab.research.google.com/github/Mansish01/Tech_Army/blob/main/hate_speech_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import cv2
import matplotlib.pyplot as plt
!pip install opendatasets
%matplotlib inline
import opendatasets as od
import PIL.Image as Image
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, UpSampling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import os
import tensorflow as tf
import datetime, os
from tensorflow.keras.callbacks import TensorBoard
from tqdm import tqdm
from numpy.linalg import norm
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
od.download("https://www.kaggle.com/datasets/mrmorj/hate-speech-and-offensive-language-dataset")

Skipping, found downloaded files in "./hate-speech-and-offensive-language-dataset" (use force=True to force download)


In [ ]:
df = pd.read_csv("/content/hate-speech-and-offensive-language-dataset/labeled_data.csv")

In [ ]:
df.head()

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [ ]:
df['tweet'][8000]

"Best behaved kids ever at mommy's commercial audition. Oreo McFlurries for everybody! #proudmom&#8230; http://t.co/9HFNSmaLcU"

In [ ]:
df_real = df[['class','tweet']]

In [ ]:
df_real['class'].value_counts()

1    19190
2     4163
0     1430
Name: class, dtype: int64

In [ ]:
df_real = df_real.loc[df_real["class"] != 0]
df_1 = df_real.loc[df_real["class"] == 2]


In [ ]:
for i in range(6):

  df_real = pd.concat([df_real,df_1])

In [ ]:
df_real['class'].value_counts()

2    29141
1    19190
Name: class, dtype: int64

In [ ]:
df

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...
...,...,...,...,...,...,...,...
24778,25291,3,0,2,1,1,you's a muthaf***in lie &#8220;@LifeAsKing: @2...
24779,25292,3,0,1,2,2,"you've gone and broke the wrong heart baby, an..."
24780,25294,3,0,3,0,1,young buck wanna eat!!.. dat nigguh like I ain...
24781,25295,6,0,6,0,1,youu got wild bitches tellin you lies


In [ ]:
df_real

,class,tweet
0,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...
...,...,...
24736,2,yaya ho.. cute avi tho RT @ViVaLa_Ari I had no...
24737,2,yea so about @N_tel 's new friend.. all my fri...
24767,2,"you know what they say, the early bird gets th..."
24779,2,"you've gone and broke the wrong heart baby, an..."


In [ ]:
#Data Preprocessing

In [ ]:
import re
import nltk
nltk.download('stopwords')
stemmer = nltk.SnowballStemmer("english")
from nltk.corpus import stopwords
import string
stopword=set(stopwords.words('english'))



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:

def clean_text(text):
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = [word for word in text.split(' ') if word not in stopword]
    text=" ".join(text)
    text = [stemmer.stem(word) for word in text.split(' ')]
    text=" ".join(text)
    return text



In [ ]:
df_real['tweet']=df_real['tweet'].apply(clean_text)

In [ ]:
df_real['tweet']

0         rt mayasolov woman shouldnt complain clean ho...
1         rt  boy dat coldtyga dwn bad cuffin dat hoe  ...
2         rt urkindofbrand dawg rt  ever fuck bitch sta...
3                   rt cganderson vivabas look like tranni
4         rt shenikarobert shit hear might true might f...
                               ...                        
24736        yaya ho cute avi tho rt vivalaari idea sleep 
24737    yea ntel new friend friend kno theyr allow one...
24767    know say earli bird get worm put gummi worm mo...
24779    youv gone broke wrong heart babi drove redneck...
24782    ruffl  ntac eileen dahlia  beauti color combin...
Name: tweet, Length: 48331, dtype: object

In [ ]:
df_real['tweet'][1]

' rt  boy dat coldtyga dwn bad cuffin dat hoe  place'

In [ ]:
df_real

,class,tweet
0,2,rt mayasolov woman shouldnt complain clean ho...
1,1,rt boy dat coldtyga dwn bad cuffin dat hoe ...
2,1,rt urkindofbrand dawg rt ever fuck bitch sta...
3,1,rt cganderson vivabas look like tranni
4,1,rt shenikarobert shit hear might true might f...
...,...,...
24736,2,yaya ho cute avi tho rt vivalaari idea sleep
24737,2,yea ntel new friend friend kno theyr allow one...
24767,2,know say earli bird get worm put gummi worm mo...
24779,2,youv gone broke wrong heart babi drove redneck...


In [ ]:
x = df_real['tweet']
y = df_real['class']

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=42)


print(len(x_train), len(y_train))
print(len(x_test), len(y_test))
type(x_train)



36248 36248
12083 12083


pandas.core.series.Series

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer(stop_words='english', ngram_range=(1,5))
x_train_vectorizer=count.fit_transform(x_train)

In [ ]:
x_test_vectorizer=count.transform(x_test)
x_train_vectorizer.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf = TfidfTransformer()

x_train_tfidf = tfidf.fit_transform(x_train_vectorizer)

x_train_tfidf.toarray()
x_test_tfidf = tfidf.transform(x_test_vectorizer)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [ ]:
model= MultinomialNB().fit(x_train_vectorizer, y_train)
prediction_vectorizer=model.predict(x_test_vectorizer)
print(confusion_matrix(y_test,prediction_vectorizer))
print (classification_report(y_test, prediction_vectorizer))

[[4521  291]
 [  20 7251]]
              precision    recall  f1-score   support

           1       1.00      0.94      0.97      4812
           2       0.96      1.00      0.98      7271

    accuracy                           0.97     12083
   macro avg       0.98      0.97      0.97     12083
weighted avg       0.98      0.97      0.97     12083



In [ ]:
import pickle

In [ ]:
with open('multinomialNB_model.pkl', 'wb') as file:
    pickle.dump(model, file)

In [113]:
test = 'hello everyone we all s'
test=[clean_text(test)]
print(test)

['hello everyon']


In [114]:
p = count.transform(test)


In [115]:
p.toarray()

array([[0, 0, 0, ..., 0, 0, 0]])

In [116]:
k = tfidf.transform(p)

In [117]:
t = model.predict(k)

In [118]:
t

array([2])

In [ ]:
/content/tokenizer.pickle